### Spark Session Build and File Opening

In [3]:
file_path = "/home/jovyan/work/data/raw/farmers-protest-tweets-2021-2-4.json"

In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("FarmersProtestTweets").getOrCreate()

In [5]:
df = spark.read.json(file_path)

### Selecting data to be stored in GCS

In [40]:
df_transformed = df.select(['id', 'user.username', 'date', 'content'])

### Transforming date to yyyy-mm-dd

In [44]:
from pyspark.sql.functions import to_date


# creating formated date column
df_transformed = df_transformed.withColumn("date_transformed", to_date(df_transformed["date"]))
df_transformed = df_transformed.drop("date")

# renaming column "date_transformed" to "date"
df_transformed = df_transformed.withColumnRenamed("date_transformed", "date")



In [45]:
df_transformed.printSchema()

root
 |-- id: long (nullable = true)
 |-- username: string (nullable = true)
 |-- content: string (nullable = true)
 |-- date: date (nullable = true)



In [46]:
df_transformed.show(5)

+-------------------+---------------+--------------------+----------+
|                 id|       username|             content|      date|
+-------------------+---------------+--------------------+----------+
|1364506249291784198|ArjunSinghPanam|The world progres...|2021-02-24|
|1364506237451313155|     PrdeepNain|#FarmersProtest \...|2021-02-24|
|1364506195453767680| parmarmaninder|ਪੈਟਰੋਲ ਦੀਆਂ ਕੀਮਤਾ...|2021-02-24|
|1364506167226032128|  anmoldhaliwal|@ReallySwara @roh...|2021-02-24|
|1364506144002088963|     KotiaPreet|#KisanEktaMorcha ...|2021-02-24|
+-------------------+---------------+--------------------+----------+
only showing top 5 rows

